In [1]:
import numpy as np
import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download('wordnet')

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wk5user3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wk5user3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
amazondf = pd.read_csv("amazon.csv", error_bad_lines=False)
amazondf

/tmp/ipykernel_63069/2613391797.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  amazondf = pd.read_csv("amazon.csv", error_bad_lines=False)


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the me

In [3]:
amazondf["discounted_price"] = amazondf["discounted_price"].str.replace("₹", "")
amazondf["discounted_price"] = amazondf["discounted_price"].str.replace(",", "")
amazondf["discounted_price"] = amazondf["discounted_price"].astype("float32")

amazondf["actual_price"] = amazondf["actual_price"].str.replace("₹", "")
amazondf["actual_price"] = amazondf["actual_price"].str.replace(",", "")
amazondf["actual_price"] = amazondf["actual_price"].astype("float32")

amazondf.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [4]:
amazondf.drop_duplicates(inplace = True)

In [5]:
pricedf = amazondf[["product_id", "product_name", "category", "about_product", "discounted_price", "actual_price"]].copy()
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,"Compatible with all Type C enabled devices, be...",199.0,349.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0


In [6]:
# pricedf['category'] = pricedf['category'].apply(lambda x: " ".join(x.split('|')))

# obtain the last category from the dataset
pricedf["category"] = pricedf['category'].apply(lambda x: x[(x.rfind('|'))+1:])
pricedf["category"].value_counts()
# pricedf.head()

USBCables             233
SmartWatches           76
Smartphones            68
SmartTelevisions       63
In-Ear                 52
                     ... 
InternalHardDrives      1
WoodenPencils           1
BatteryChargers         1
DataCards&Dongles       1
HandheldBags            1
Name: category, Length: 207, dtype: int64

In [7]:
# create 7 descriptors and compare their results

pricedf["description1"] = pricedf["product_name"] 
pricedf["description2"] = pricedf["about_product"]
pricedf["description3"] = pricedf["category"]
pricedf["description4"] = pricedf["product_name"] + " " + pricedf["about_product"]
pricedf["description5"] = pricedf["product_name"] + " " + pricedf["category"]
pricedf["description6"] = pricedf["about_product"] + " " + pricedf["category"]
pricedf["description7"] = pricedf["product_name"] + " " + pricedf["about_product"] + " " + pricedf["category"]
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description1,description2,description3,description4,description5,description6,description7
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,USBCables,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,USBCables,Wayona Nylon Braided USB to Lightning Fast Cha...,Wayona Nylon Braided USB to Lightning Fast Cha...,High Compatibility : Compatible With iPhone 12...,Wayona Nylon Braided USB to Lightning Fast Cha...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,USBCables,"Compatible with all Type C enabled devices, be...",199.0,349.0,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",USBCables,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,"Compatible with all Type C enabled devices, be...",Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,USBCables,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,USBCables,Sounce Fast Phone Charging Cable & Data Sync U...,Sounce Fast Phone Charging Cable & Data Sync U...,【 Fast Charger& Data Sync】-With built-in safet...,Sounce Fast Phone Charging Cable & Data Sync U...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,USBCables,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,USBCables,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,The boAt Deuce USB 300 2 in 1 cable is compati...,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,USBCables,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,USBCables,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,Portronics Konnect L 1.2M Fast Charging 3A 8 P...


In [8]:
def remove_stopwords(sentence):
    stop = set(stopwords.words('english'))
    processed = " ".join([word for word in sentence.split() if word not in stop])
    return processed

def remove_urls(sentence):
    processed = re.sub('http[s]?://(?:[a-zA-Z]|[0–9]|[$-_@.&+#]|[!*\(\),]|'\
                   '(?:%[0–9a-fA-F][0–9a-fA-F]))+','', sentence)
    return processed

def remove_punctuation(sentence):
    punc = string.punctuation
    sentence = sentence.translate(str.maketrans('', '', punc))
    return sentence


In [9]:
lst = ["description1", "description2", "description3", "description4", "description5", "description6", "description7"]

for description in lst:
    pricedf[description] = pricedf[description].apply(lambda x: remove_urls(x))
    pricedf[description] = pricedf[description].apply(lambda x: x.lower())
    pricedf[description] = pricedf[description].apply(lambda x: remove_stopwords(x))
    pricedf[description] = pricedf[description].apply(lambda x: remove_punctuation(x))

    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: remove_urls(x))
    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: x.lower())
    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: remove_stopwords(x))
    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: remove_punctuation(x))

# pricedf.head()

lemmatizer = WordNetLemmatizer()
def lemmatization(sentence):
    sentence = ' '.join([lemmatizer.lemmatize(word) for word in sentence.split()])
    return sentence

for description in lst:
    pricedf[description] = pricedf[description].apply(lambda x: lemmatization(x))
# pricedf["name_description"] = pricedf["name_description"].apply(lambda x: lemmatization(x))
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description1,description2,description3,description4,description5,description6,description7
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,USBCables,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,wayona nylon braided usb lightning fast chargi...,high compatibility compatible iphone 12 11 xxs...,usbcables,wayona nylon braided usb lightning fast chargi...,wayona nylon braided usb lightning fast chargi...,high compatibility compatible iphone 12 11 xxs...,wayona nylon braided usb lightning fast chargi...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,USBCables,"Compatible with all Type C enabled devices, be...",199.0,349.0,ambrane unbreakable 60w 3a fast charging 15m b...,compatible type c enabled device android smart...,usbcables,ambrane unbreakable 60w 3a fast charging 15m b...,ambrane unbreakable 60w 3a fast charging 15m b...,compatible type c enabled device android smart...,ambrane unbreakable 60w 3a fast charging 15m b...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,USBCables,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,sounce fast phone charging cable data sync usb...,【 fast charger data sync】with builtin safety p...,usbcables,sounce fast phone charging cable data sync usb...,sounce fast phone charging cable data sync usb...,【 fast charger data sync】with builtin safety p...,sounce fast phone charging cable data sync usb...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,USBCables,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,boat deuce usb 300 2 1 typec micro usb stress ...,boat deuce usb 300 2 1 cable compatible smartp...,usbcables,boat deuce usb 300 2 1 typec micro usb stress ...,boat deuce usb 300 2 1 typec micro usb stress ...,boat deuce usb 300 2 1 cable compatible smartp...,boat deuce usb 300 2 1 typec micro usb stress ...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,USBCables,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,portronics konnect l 12m fast charging 3a 8 pi...,charge sync function cable come charging data ...,usbcables,portronics konnect l 12m fast charging 3a 8 pi...,portronics konnect l 12m fast charging 3a 8 pi...,charge sync function cable come charging data ...,portronics konnect l 12m fast charging 3a 8 pi...


In [10]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import Ridge


results1 = []
results2 = []
for description in lst:
    result1 = [description]
    result2 = [description]
    X_train, X_test, y_train, y_test = train_test_split(pricedf[description], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

    print(description)
    tv = TfidfVectorizer()
    # train_tfidf = tv.fit_transform(ratingdf["combined_review"]).toarray()
    X_train_tfidf = tv.fit_transform(X_train) # fit train
    X_test_tfidf = tv.transform(X_test) # transform test
    print(X_train_tfidf.shape, X_test_tfidf.shape)


    print("----- linear regression -----")
    linear = LinearRegression()
    linear = make_pipeline(StandardScaler(with_mean=False), linear)
    linear.fit(X_train_tfidf, y_train)
    y_pred = linear.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))
    print(linear.score(X_test_tfidf, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(linear.score(X_test_tfidf, y_test))
    # print(y_pred[:10])
    # print(y_test[:10])


    print("\n ----- support vector regression linear -----")
    svr = SVR(kernel='linear')
    svr = make_pipeline(StandardScaler(with_mean=False), svr)
    svr.fit(X_train_tfidf, y_train)
    print(svr.score(X_test_tfidf, y_test))

    y_pred = svr.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(svr.score(X_test_tfidf, y_test))

    
    print("\n ----- support vector regression rbf -----")
    svr = SVR(kernel='rbf')
    svr = make_pipeline(StandardScaler(with_mean=False), svr)
    svr.fit(X_train_tfidf, y_train)
    print(svr.score(X_test_tfidf, y_test))

    y_pred = svr.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(svr.score(X_test_tfidf, y_test))

    # print(y_pred[:10])
    # print(y_test[:10])


    print("\n ----- ridge regression -----")
    ridge = Ridge(alpha = 0.5)
    ridge.fit(X_train_tfidf, y_train)
    print(ridge.score(X_test_tfidf, y_test))

    y_pred = ridge.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(ridge.score(X_test_tfidf, y_test))

    # print(y_pred[:10])
    # print(y_test[:10])

    print("\n")

    results1.append(result1)
    results2.append(result2)

description1
(1172, 3936) (293, 3936)
----- linear regression -----
0.8685527203956477
0.8859393485935034

 ----- support vector regression linear -----
0.3609988852317315
-4.152239319850285

 ----- support vector regression rbf -----
-0.12953288211411218
-2876008.428418382

 ----- ridge regression -----
0.8994022649365694
0.8942074619126358


description2
(1172, 12540) (293, 12540)
----- linear regression -----
0.7524163319191358
0.7941264276430353

 ----- support vector regression linear -----
0.7791431935092479
0.5088984025308789

 ----- support vector regression rbf -----
-0.12976506473074867
-2856668.889893657

 ----- ridge regression -----
0.8296924057503894
0.7726539879238068


description3
(1172, 195) (293, 195)
----- linear regression -----
0.6836554626559403
0.7855063384005602

 ----- support vector regression linear -----
-0.043132353933705625
-251.93804254644994

 ----- support vector regression rbf -----
-0.12871310938419378
-2918195.2865464133

 ----- ridge regression ---

In [11]:
from tabulate import tabulate


headers = ["Descriptor", "linear regression", "svr linear", "svr rbf", "ridge"] 

table1 = tabulate(results1, headers=headers, tablefmt="fancy_grid")
print(table1)

table2 = tabulate(results2, headers=headers, tablefmt="fancy_grid")
print(table2)

╒══════════════╤═════════════════════╤══════════════╤══════════════╤══════════╕
│ Descriptor   │   linear regression │   svr linear │      svr rbf │    ridge │
╞══════════════╪═════════════════════╪══════════════╪══════════════╪══════════╡
│ description1 │            0.868553 │    -4.15224  │ -2.87601e+06 │ 0.894207 │
├──────────────┼─────────────────────┼──────────────┼──────────────┼──────────┤
│ description2 │            0.752416 │     0.508898 │ -2.85667e+06 │ 0.772654 │
├──────────────┼─────────────────────┼──────────────┼──────────────┼──────────┤
│ description3 │            0.683655 │  -251.938    │ -2.9182e+06  │ 0.671779 │
├──────────────┼─────────────────────┼──────────────┼──────────────┼──────────┤
│ description4 │            0.785167 │     0.559343 │ -3.01488e+06 │ 0.814703 │
├──────────────┼─────────────────────┼──────────────┼──────────────┼──────────┤
│ description5 │            0.89386  │    -3.33859  │ -3.2941e+06  │ 0.909388 │
├──────────────┼─────────────────────┼──

In [26]:
description = "description5"
result1 = [description]
result2 = [description]
X_train, X_test, y_train, y_test = train_test_split(pricedf[description], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

print(description)
tv = TfidfVectorizer()
# train_tfidf = tv.fit_transform(ratingdf["combined_review"]).toarray()
X_train_tfidf = tv.fit_transform(X_train) # fit train
X_test_tfidf = tv.transform(X_test) # transform test
print(X_train_tfidf.shape, X_test_tfidf.shape)


print("----- linear regression -----")
linear = LinearRegression()
linear = make_pipeline(StandardScaler(with_mean=False), linear)
linear.fit(X_train_tfidf, y_train)
y_pred = linear.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))
print(linear.score(X_test_tfidf, y_test))

result1.append(r2_score(y_pred, y_test))
result2.append(linear.score(X_test_tfidf, y_test))
print(y_pred[:10])
print(y_test[:10])


print("\n ----- support vector regression linear -----")
svr = SVR(kernel='linear')
svr = make_pipeline(StandardScaler(with_mean=False), svr)
svr.fit(X_train_tfidf, y_train)
print(svr.score(X_test_tfidf, y_test))

y_pred = svr.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))

result1.append(r2_score(y_pred, y_test))
result2.append(svr.score(X_test_tfidf, y_test))

print(list(X_test[:10]))
print(y_pred[:10])
print(y_test[:10])


print("\n ----- ridge regression -----")
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train_tfidf, y_train)
print(ridge.score(X_test_tfidf, y_test))

y_pred = ridge.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))

result1.append(r2_score(y_pred, y_test))
result2.append(ridge.score(X_test_tfidf, y_test))

print(y_pred[:10])
print(y_test[:10])

print("\n")

results1.append(result1)
results2.append(result2)

description5
(1172, 4100) (293, 4100)
----- linear regression -----
0.8938598102679834
0.9051960004657696
[20385.8698157   -803.18928145 27334.12703253  5467.51340018
  5080.51832576   746.68981192  1491.89301805  1612.58211641
    84.88162604  3002.85170151]
434     20999.0
194       599.0
82      19125.0
294       899.0
1313     4495.0
1268      980.0
317       399.0
613      1995.0
320       599.0
117       999.0
Name: actual_price, dtype: float32

 ----- support vector regression linear -----
0.40188158766441395
-3.338585126737734
['iqoo z6 44w vivo lumen blue 6gb ram 128gb storage 644 fhd amoled display 50 charge 27 min indisplay fingerprint scanning smartphones', 'duracell micro usb 3a braided sync fast charging cable 39 foot 12m support qc 2030 charging high speed data transmission black usbcables', 'skywall 8128 cm 32 inch hd ready smart led tv 32swelspro black smarttelevisions', 'amazonbasics usb typec microb 20 cable 6 inch 152 centimeter white usbcables', 'inalsa upright vac

Predictor Function for test

In [55]:
# descriptor 5 with ridge regression is the best

rand_idx = np.random.randint(0, len(pricedf), )
test_desc = pricedf.iloc[rand_idx]["description5"]
X_test_tfidf = tv.transform([test_desc])
pred = ridge.predict(X_test_tfidf)

def insert_newline_every_n_chars(text, n=10):
    new_text = ''
    for i, char in enumerate(text):
        new_text += char
        if (i + 1) % n == 0 and i != len(text) - 1:
            new_text += '\n'
    return new_text


test_desc = insert_newline_every_n_chars(test_desc, 60)

print("test description: ", test_desc)
print("predicted price: ", pred[0])
print("actual price: ", pricedf.iloc[rand_idx]["actual_price"])


test description:  redmi note 11 space black 6gb ram 64gb storage 90hz fhd amol
ed display qualcomm® snapdragon™ 6806nm 33w charger included
 smartphones
predicted price:  18836.941458365527
actual price:  18999.0


# BERT

X_train, X_test, y_train, y_test = train_test_split(pricedf["description"], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

In [13]:
# pricedf.head()

pricedf['product_name'].head(10)

0    Wayona Nylon Braided USB to Lightning Fast Cha...
1    Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2    Sounce Fast Phone Charging Cable & Data Sync U...
3    boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4    Portronics Konnect L 1.2M Fast Charging 3A 8 P...
5    pTron Solero TB301 3A Type-C Data and Fast Cha...
6    boAt Micro USB 55 Tangle-free, Sturdy Micro US...
7               MI Usb Type-C Cable Smartphone (Black)
8    TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...
9    Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
Name: product_name, dtype: object

In [14]:
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('all-mpnet-base-v2')

2023-05-07 07:49:36.852600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 07:49:36.897128: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 07:49:36.897869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 07:49:37.468130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
# flatten text list
flatten_list = lambda y:[x for a in y for x in flatten_list(a)] if type(y) is list else [y]
description_list = flatten_list(pricedf['description5'].values.tolist())
price_list = flatten_list(pricedf['actual_price'].values.tolist())

print(description_list[:10])


['wayona nylon braided usb lightning fast charging data sync cable compatible iphone 13 1211 x 8 7 6 5 ipad air pro mini 3 ft pack 1 grey usbcables', 'ambrane unbreakable 60w 3a fast charging 15m braided type c cable smartphones tablet laptop type c device pd technology 480mbps data sync quick charge 30 rct15a black usbcables', 'sounce fast phone charging cable data sync usb cable compatible iphone 13 1211 x 8 7 6 5 ipad air pro mini io device usbcables', 'boat deuce usb 300 2 1 typec micro usb stress resistant tanglefree sturdy cable 3a fast charging 480mbps data transmission 10000 bend lifespan extended 15m lengthmartian red usbcables', 'portronics konnect l 12m fast charging 3a 8 pin usb cable charge sync function iphone ipad grey usbcables', 'ptron solero tb301 3a typec data fast charging cable made india 480mbps data sync strong durable 15meter nylon braided usb cable typec device charging adapter black usbcables', 'boat micro usb 55 tanglefree sturdy micro usb cable 3a fast charg

In [16]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import Ridge


results1 = []
results2 = []
for description in lst:
    result1 = [description]
    result2 = [description]

    # flatten text list
    flatten_list = lambda y:[x for a in y for x in flatten_list(a)] if type(y) is list else [y]
    description_list = flatten_list(pricedf[description].values.tolist())
    price_list = flatten_list(pricedf['actual_price'].values.tolist())

    # create embeddings for bert
    description_embeddings = model.encode(description_list)

    # X_train, X_test, y_train, y_test = train_test_split(pricedf[description], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)
    X_train, X_test, y_train, y_test = train_test_split(description_embeddings, price_list,  test_size=0.2, random_state=11, shuffle=True, stratify=None)

    


    print("----- linear regression -----")
    linear = LinearRegression()
    linear = make_pipeline(StandardScaler(with_mean=False), linear)
    linear.fit(X_train, y_train)
    y_pred = linear.predict(X_test)
    print(r2_score(y_pred, y_test))
    print(linear.score(X_test, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(linear.score(X_test, y_test))
    print(y_pred[:10])
    print(y_test[:10])


    print("\n ----- support vector regression -----")
    svr = SVR(kernel='linear')
    svr = make_pipeline(StandardScaler(with_mean=False), svr)
    svr.fit(X_train, y_train)
    print(svr.score(X_test, y_test))

    y_pred = svr.predict(X_test)
    print(r2_score(y_pred, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(svr.score(X_test, y_test))

    print(y_pred[:10])
    print(y_test[:10])


    print("\n ----- ridge regression -----")
    ridge = Ridge(alpha = 0.5)
    ridge.fit(X_train, y_train)
    print(ridge.score(X_test, y_test))

    y_pred = ridge.predict(X_test)
    print(r2_score(y_pred, y_test))

    result1.append(r2_score(y_pred, y_test))
    result2.append(ridge.score(X_test, y_test))

    print(y_pred[:10])
    print(y_test[:10])

    print("\n")

    results1.append(result1)
    results2.append(result2)

----- linear regression -----
0.561199491900179
0.28433438308838854
[24075.8     7399.168  47939.91   -7280.9453  1426.9453 22335.598
  6503.3867 -3342.1719 22443.68    2222.129 ]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]

 ----- support vector regression -----
0.34867469252536276
-5.245128133155242
[14066.47431015  -145.42468385 11427.03875249   516.79487653
  3614.69428661  4011.43099897  -902.09993099   268.9148206
  1806.48692696  1669.97903173]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]

 ----- ridge regression -----
0.7810584228671879
0.7290900717535618
[23489.328    3343.2825  30583.355     831.8032   8399.64     9630.633
 -4700.6895    986.1587   3045.8687    -46.70117]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]


----- linear regression -----
0.36035002476372835
-0.25201439464728836
[ 19537.8      9457.454   36349.54    -2576.1953  -4519.627   -3982.287
 -28860.334    7905.95

In [17]:
from tabulate import tabulate

headers = ["Descriptor", "linear regression", "svr", "ridge"] 

table1 = tabulate(results1, headers=headers, tablefmt="fancy_grid")
print(table1)

table2 = tabulate(results2, headers=headers, tablefmt="fancy_grid")
print(table2)

╒══════════════╤═════════════════════╤══════════╤══════════╕
│ Descriptor   │   linear regression │      svr │    ridge │
╞══════════════╪═════════════════════╪══════════╪══════════╡
│ description1 │         0.561199    │ -5.24513 │ 0.72909  │
├──────────────┼─────────────────────┼──────────┼──────────┤
│ description2 │         0.36035     │ -4.39459 │ 0.575034 │
├──────────────┼─────────────────────┼──────────┼──────────┤
│ description3 │        -0.000755345 │ -1.33872 │ 0.641663 │
├──────────────┼─────────────────────┼──────────┼──────────┤
│ description4 │         0.49093     │ -3.66215 │ 0.680713 │
├──────────────┼─────────────────────┼──────────┼──────────┤
│ description5 │         0.53059     │ -4.18513 │ 0.744955 │
├──────────────┼─────────────────────┼──────────┼──────────┤
│ description6 │         0.344001    │ -4.24103 │ 0.584136 │
├──────────────┼─────────────────────┼──────────┼──────────┤
│ description7 │         0.523312    │ -3.62023 │ 0.682745 │
╘══════════════╧════════

In [18]:
# database sentences
description_embeddings = model.encode(description_list)
print(description_embeddings.shape)

# faq sentences
# faq_embeddings = model.encode(faq_list)
# print(faq_embeddings.shape)

(1465, 768)


In [19]:
description = "description5"
result1 = [description]
result2 = [description]
# X_train, X_test, y_train, y_test = train_test_split(pricedf[description], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)
X_train, X_test, y_train, y_test = train_test_split(description_embeddings, price_list,  test_size=0.2, random_state=11, shuffle=True, stratify=None)



print(description)
# tv = TfidfVectorizer()
# # train_tfidf = tv.fit_transform(ratingdf["combined_review"]).toarray()
# X_train_tfidf = tv.fit_transform(X_train) # fit train
# X_test_tfidf = tv.transform(X_test) # transform test
# print(X_train_tfidf.shape, X_test_tfidf.shape)


print("----- linear regression -----")
linear = LinearRegression()
linear = make_pipeline(StandardScaler(with_mean=False), linear)
linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)
print(r2_score(y_pred, y_test))
print(linear.score(X_test, y_test))

result1.append(r2_score(y_pred, y_test))
result2.append(linear.score(X_test, y_test))
print(y_pred[:10])
print(y_test[:10])


print("\n ----- support vector regression -----")
svr = SVR(kernel='linear')
svr = make_pipeline(StandardScaler(with_mean=False), svr)
svr.fit(X_train, y_train)
print(svr.score(X_test, y_test))

y_pred = svr.predict(X_test)
print(r2_score(y_pred, y_test))

result1.append(r2_score(y_pred, y_test))
result2.append(svr.score(X_test, y_test))

print(y_pred[:10])
print(y_test[:10])


print("\n ----- ridge regression -----")
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
print(ridge.score(X_test, y_test))

y_pred = ridge.predict(X_test)
print(r2_score(y_pred, y_test))

result1.append(r2_score(y_pred, y_test))
result2.append(ridge.score(X_test, y_test))

print(y_pred[:10])
print(y_test[:10])

print("\n")

results1.append(result1)
results2.append(result2)

description5
----- linear regression -----
0.5233121521661795
0.20904267551546096
[21237.695   -1872.6211  47510.1     -4490.3477  -5501.9023    600.23047
 -2793.9766  18711.691   -2751.5664   8568.41   ]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]

 ----- support vector regression -----
0.40924604872619663
-3.620230354535331
[14005.99935804   -59.18919898 12142.71460896  1384.99896935
  3621.41469799   956.70024166 -2351.08255064 -1300.15206566
  2443.38289359  2620.05871954]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]

 ----- ridge regression -----
0.766605307065536
0.6827448419376849
[25323.617  -1241.5381 30182.078    976.7776  8573.764   4475.475
 -9064.47   -2489.7192   581.4756  3238.621 ]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]




In [20]:
# X_train, X_test, y_train, y_test = train_test_split(p, pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(description_embeddings, price_list,  test_size=0.2, random_state=11, shuffle=True, stratify=None)

print(X_train_bert.shape, X_test_bert.shape)
print(len(y_train_bert), len(y_test_bert))

(1172, 768) (293, 768)
1172 293


In [21]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train_bert, y_train_bert)
y_pred = linear.predict(X_test_bert)
print(r2_score(y_pred, y_test_bert))

print(linear.score(X_test_bert, y_test_bert))

print(y_pred[:10])
print(y_test_bert[:10])


0.5033427455623655
0.17080347986230726
[21349.637   -1622.3853  46816.89    -3894.6548  -5418.487     220.05713
 -4039.026   19302.635    -584.8618   7776.195  ]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]


In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
sc = StandardScaler()
# X_train_sc = sc.fit_transform(X_train_tfidf)
# y_train_sc = sc.fit_transform(y_train)
# X_test_sc = sc.transform(X_test_tfidf)
# y_test_sc = sc.transform(y_test)

svr = SVR(kernel='linear')
svr.fit(X_train_bert, y_train_bert)
y_pred = svr.predict(X_test_bert)
print(r2_score(y_pred, y_test_bert))

print(svr.score(X_test_bert, y_test_bert))

print(y_pred[:10])
print(y_test_bert[:10])

-69903.48407989532
-0.12638362738824305
[1693.49906889 1559.74219497 1694.60119864 1566.35367086 1657.72248601
 1658.15064494 1623.99928566 1597.71142091 1616.42903603 1583.761048  ]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]


In [23]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train_bert, y_train_bert)
y_pred = ridge.predict(X_test_bert)
print(r2_score(y_pred, y_test_bert))

print(ridge.score(X_test_bert, y_test_bert))

print(y_pred[:10])
print(y_test_bert[:10])

0.6827448419376849
0.766605307065536
[25323.617  -1241.5381 30182.078    976.7776  8573.764   4475.475
 -9064.47   -2489.7192   581.4756  3238.621 ]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]
